## Importing Packages

In [ ]:
import numpy as np
import pandas as pd

## Reading-In and Wrangling Data

In [ ]:
df_spy = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "spy")
df_spy

,date,spy
0,2000-01-03,92.692932
1,2000-01-04,89.068039
2,2000-01-05,89.227402
3,2000-01-06,87.793404
4,2000-01-07,92.892082
...,...,...
6284,2024-12-24,601.299988
6285,2024-12-26,601.340027
6286,2024-12-27,595.010010
6287,2024-12-30,588.219971


## Calculating Returns and Equity Curves

In [ ]:
df_spy["ret_spy"] = df_spy["spy"].pct_change()
df_spy.fillna(0, inplace=True)

In [ ]:
df_spy["equity_spy"] = (1 + df_spy["ret_spy"]).cumprod()
df_spy["drawdown_spy"] = (df_spy["equity_spy"] / df_spy["equity_spy"].cummax()) - 1
df_spy

,date,spy,ret_spy,equity_spy,drawdown_spy
0,2000-01-03,92.692932,0.000000,1.000000,0.000000
1,2000-01-04,89.068039,-0.039106,0.960894,-0.039106
2,2000-01-05,89.227402,0.001789,0.962613,-0.037387
3,2000-01-06,87.793404,-0.016071,0.947142,-0.052858
4,2000-01-07,92.892082,0.058076,1.002148,0.000000
...,...,...,...,...,...
6284,2024-12-24,601.299988,0.011115,6.487010,-0.007381
6285,2024-12-26,601.340027,0.000067,6.487442,-0.007315
6286,2024-12-27,595.010010,-0.010527,6.419152,-0.017764
6287,2024-12-30,588.219971,-0.011412,6.345899,-0.028973


## Determining Drawdown Period Start and End Dates

Notice that this is a bit misleading, because two days of consecutive all time highs will show up as a "drawdown period".  However, when a I do the query for actual corrections, this will be take care of.

In [ ]:
start_dates = df_spy.query("drawdown_spy == 0")["date"].iloc[:-1].values
end_dates = df_spy.query("drawdown_spy == 0")["date"].iloc[1:]

In [ ]:
df_drawdown_periods = pd.DataFrame({
    "start_date":start_dates,
    "end_date":end_dates,
}).reset_index(drop=True)
df_drawdown_periods

,start_date,end_date
0,2000-01-03,2000-01-07
1,2000-01-07,2000-01-10
2,2000-01-10,2000-01-14
3,2000-01-14,2000-01-19
4,2000-01-19,2000-03-17
...,...,...
544,2024-11-26,2024-11-29
545,2024-11-29,2024-12-02
546,2024-12-02,2024-12-03
547,2024-12-03,2024-12-04


## Calculating the Maximum Drawdowns during Drawdown Period

In [ ]:
period_drawdowns = []
bottom_dates = []
for ix in df_drawdown_periods.index:
    start_date = df_drawdown_periods.at[ix, "start_date"]
    end_date = df_drawdown_periods.at[ix, "end_date"]
    drawdown = df_spy.query("@start_date <= date & date <= @end_date")["drawdown_spy"].min()
    df = df_spy.query("@start_date <= date & date <= @end_date")
    bottom_dt = df[df["drawdown_spy"] == drawdown]["date"].iloc[0]
    period_drawdowns.append(drawdown)
    bottom_dates.append(bottom_dt)

In [ ]:
df_drawdown_periods["bottom_date"] = bottom_dates
df_drawdown_periods["drawdown_spy"] = period_drawdowns

## Filtering for 5% Corrections

In [ ]:
df_corrections = df_drawdown_periods.query("drawdown_spy < -0.05")
df_corrections

,start_date,end_date,bottom_date,drawdown_spy
0,2000-01-03,2000-01-07,2000-01-06,-0.052858
4,2000-01-19,2000-03-17,2000-02-25,-0.093006
9,2000-03-24,2006-10-26,2002-10-09,-0.475159
33,2007-02-20,2007-04-16,2007-03-05,-0.059505
51,2007-07-19,2007-10-05,2007-08-15,-0.090475
53,2007-10-09,2012-08-16,2009-03-09,-0.551894
61,2012-09-14,2013-01-02,2012-11-15,-0.073456
102,2013-05-21,2013-07-11,2013-06-24,-0.055506
133,2013-12-31,2014-02-24,2014-02-03,-0.056960
170,2014-09-18,2014-10-31,2014-10-16,-0.072734


In [ ]:
df_corrections.to_csv("spy_corrections.csv", index=False)